# Import Libraries First

In [1]:
%load_ext autoreload
%autoreload 2
from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees.asm_tree_Basic import augment_RM

augment_RM( robot )

%store -r

####### LIBS #######################################################################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector
from py_trees.decorators import FailureIsSuccess , SuccessIsFailure

# import 
from rmlib.rmtools.assembly_trees.asm_tree_Basic import *

from rmlib.rmtools.assembly_trees.asm_tree_FT_based import ( 
    Bias_Wrist , Spiral_Insert , Tilt_Insert , Spin_Press , Move_to_Contact , SpiralStep
)
    
from rmlib.rmtools.assembly_trees.asm_tree_drill import *

from rmlib.rmtools.assembly_trees.asm_tree_logic_flow import Run_to_X_Failures_DECO

import math
from pmath import *

from utils import is_matx_list
from math import radians

print( "Libs loaded!" )

from task_parameters import taskParams , _TB_side , _M8_TB_height , _M6_TB_height , _M4_TB_height

_DRILL_CONTACT_PRESS = 11.0

import pickle , traceback

def save_names_to_PKL( pklPath , nameStringList ):
    pklStruct = {}
    print( vars() )
    for name in nameStringList:
        pklStruct[ name ] = globals()[ name ]
        
    with open( pklPath , "wb" ) as f:
        try:
            print( "Pickling" , pklPath , end = '... ' )
            pickle.dump( pklStruct , f )
            print( "Success!" )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
    print( "### Completed Writing" , pklPath , "###" )
    
def load_names_from_PKL_to_global( pklPath , nameStringList ):
    n = 0
    with open( pklPath , "rb" ) as f:
        try:
            print( "Unpickling" , pklPath , end = '... ' )
            pklStruct = pickle.load( f )
            print( "Got: " , type( pklStruct ) , "with" , len( pklStruct ) , "elements" )
            globDict = globals()
            for name in nameStringList:
                try:
                    globals()[ name ] = pklStruct[ name ]
                    n += 1
                except:
                    print( "Could not load '" + str( name ) + "'! Continue ..." )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
            
    print( "### Read" , n , "vars from" , pklPath , "into global dict ###" )

load_names_from_PKL_to_global( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' ,
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)

load_names_from_PKL_to_global( 
    pklPath = "dct.pkl" , 
    nameStringList = [ "dct" ]
)


## BIAS TEST ##
if 0:
    for i in range(40):
        print('##', i+1, '##')
        before = robot.ft.get_wrist_force()
        bgn    = time.time()
        robot.ft.bias_wrist_force()
        print( time.time() - bgn , "[s] bias time" )
        after  = robot.ft.get_wrist_force()
        print( after )
        print( "Diff:" , np.linalg.norm( np.subtract( after, before ) ) , '\n\n' )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Loading {'active_components': {'hand': ['hand_config'], 'ft': ['ft_config'], 'arm': ['arm_config'], 'camera': ['cam_config']}, 'my_components': {'hand_config': {'module_name': 'hands.smarthand', 'class_name': 'SmartHand', 'finger_width_inner': 0.0, 'finger_depth': 0.014, 'finger_length': 0.0415, 'finger_width_outer': 0.015}, 'arm_config': {'xmlrpc_port': '8003', 'class_name': 'UR5', 'max_joint_speed': 1.05, 'ip_address': '192.168.0.101', 'default_linear_speed': 0.1, 'max_joint_accel': 1.4, 'default_joint_accel': 0.8, 'module_name': 'arms.ur5', 'max_linear_accel': 1.2, 'default_joint_speed': 0.7, 'default_linear_accel': 0.8, 'max_linear_speed': 0.25}, 'cam_config': {'ci_cam_offset': [-0.036, -0.028, -0.079], 'module_name': 'cameras.realsense', 'class_name': 'RealSense', 'camera_model': 'd410', 'pc_cam_offset': [-0.037, -0.033, -0.079]}, 'ft_config': {'local_port': 20

# Arm Movements
## Move to Absolute Pose `Move_Arm`

In [2]:
"""
### Move to Safe Pose ###
Basic movements are defined in:
rmlib/rmlib/rmtools/assembly_trees/asm_tree_Basic.py

All of the basic behaviors and trees inherit `py_trees` classes.
""" 

moveNode = Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB

## Run Behavior
`run_BT_until_done` is a function that ticks the node object provided as an argument at regular intervals.  It will report the final status of the root node and print and ASCII tree representation.   
The default args are likely adequate for your purpose, but it can be found at  
rmlib/rmlib/rmtools/assembly_trees/asm_tree_Basic.py

In [4]:
run_BT_until_done( moveNode )

### RUN BT Move_Arm ###

Root node Move_Arm succeeded!

--> Move_Arm [o]


Run completed! with status Status.SUCCESS after 1.5300235748291016 seconds.



### BT SESSION OVER ###


## Pose Math, Move to Contact, and Tree Building

In [7]:
# Calc a pose 10cm below the `SAFE_POSE`
lowPose = translate_pose( 
    SAFE_POSE , 
    [ 0.0 , 0.0 , -0.100 ] , # Translation vector
    dir_pose = 'origin' # { 'origin': lab frame, 'self': relative the argument pose }
)
# Location:  rmlib/rmlib/rmtools/pmath.py

The following cell builds a sequence that makes a relative move, then stores the final pose under a global dictionary key for later retrieval.  
For most 'procedure-like' trees, a Sequence with Memory is used.  
The memory option prevents completed actions from being ticked again until the entire `Sequence` has completed.  
Behaviors will be executed in the order added.

In [8]:
pushTree = Sequence( memory = 1 ) 

poseKey = "TouchdownPose" # String key to store pose that will be used by another tree

# 1. Move to a pose conveniently close to the table
pushTree.add_child(
    Move_Arm( pose = lowPose , mode = 'l',  speed = 0.125 , accel = 0.35 , ctrl = robot ) 
)

# 2. Close the fingers in prep for a probing action
pushTree.add_child(
    Set_Fingers( 
        openState = 0.0 , # Dist b/n fingers in [m], Anything larger than 0.095 interpreted as fully open
        ctrl = robot , # rmlib ref
        waitSec = 0.25 # OpenCM does not block execution, so pause to let fingers physically close
    )
)

"""
### Move Until Reaction Force Encountered ###
Force-based actions are defined in:
rmlib/rmlib/rmtools/assembly_trees/asm_tree_FT_based.py
""" 

# 3. Approach the table at a **gentle speed** and stop when a small reaction force is observed 
pushTree.add_child(
    Move_to_Contact( 
        Fmag      = 3.0 , # Reaction force that triggers stop, Forces below 1.5N aren't reliably detected
        relMove   = [0.0, 0.0, -1.0] , # Specify a move the goes *beyond* where you expect to end up
        biasWrist = 1 , # Bias wrist before beginning motion (Sorry, currently this takes 1-8 seconds)
        mode      = 'l' , 
        speed     = 0.040 , accel = 0.04 , # **gentle speed**
        pull      = 0 , # For pulling actions, reaction force tugs on hand rather than pushes
        ctrl      = robot
    ) # NOTE: This behav. may be used for absolute pose if an argument for `pose` is provided
)

# 4. Remember where we landed
pushTree.add_child(
    Store_Current_Pose( 
        keyString = poseKey , # Blackboard dict key
        ctrl = robot , 
        setOnce = True # Set this flag to prevent the value from being modified once stored
    )
)

# 5. Relative move back up, no condition
pushTree.add_child(
    Move_Arm_Relative( 
        translation = [ -0.035, -0.035, 0.070 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        mode = 'l' , speed = 0.125 , accel = 0.35 , 
        stop_cond = None , # A custon stop condition can be passed here if desired
        frame = 'origin' ,
        cond_success = 1 , # Flag true if we are using "stop_cond()==1" to mean *success*, otherwise the reverse
        bias_wrist = 0 , # If `stop_cond` is force based, this is a good idea
        ctrl = robot
    ) # NOTE: If no stop_cond is provided, the this behavior returns true only if the full motion has completed
)

# 6. Now let's recover the pose that we stored before
pushTree.add_child(
    Move_Arm( BB_key = poseKey , # Blackboard dict key passed to `BB_key` instead of a `pose` arg
              mode = 'l',  speed = 0.063 , accel = 0.17 , ctrl = robot ) 
)

# 7. It's a good idea to return to a known safe pose between activities
pushTree.add_child(
    Move_Arm( pose  = SAFE_POSE , ctrl = robot )
)

# 8. Open the fingers
pushTree.add_child(
    Set_Fingers( openState = 1.0 , ctrl = robot , waitSec = 0.0 )
)

print()

You have created something beautiful, and you only wish to see your creation flourish:

In [9]:
run_BT_until_done( pushTree )

### RUN BT Sequence ###

about to bias
about to fetch
get reading
bias complete!
Root node Sequence succeeded!

[-] Sequence [o]
    --> Move_Arm [o]
    --> Set_Fingers [o]
    --> Move_to_Contact [o]
    --> Store_Current_Pose [o]
    --> Move_Arm_Relative [o]
    --> Move_Arm [o]
    --> Move_Arm [o]
    --> Set_Fingers [o]


Run completed! with status Status.SUCCESS after 16.47796130180359 seconds.



### BT SESSION OVER ###


# Drill Operations
### **Mind drill clearance, it is very tall now!**
### Do not keep drill grasped for too long, hand overheats!

In [16]:
# Adjust this up in increments of 0.5-1.0 if you are having trouble picking up drill
_DRILL_CONTACT_PRESS = 11.0 # Too aggressive and holders move out of place over time

_LONG_DRILL_SAFE_Z = 0.250 # <-- VERY IMPORTANT - AVOID BREAKING DRILL

## Take M4

In [17]:
testTree = Unholster_Drill( 
    M4_GraspPose , 
    'M4' , 
    betweenPose = midDrillPose , 
    zApproach=0.20, zClose=0.040, zRetry=0.030 , Tpliant = 0.3 , 
    wdthNarrow = 0.038 , maxIter=5, drillPress = _DRILL_CONTACT_PRESS , 
    zFree=_LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
    ctrl = robot 
)

run_BT_until_done( testTree , N = 90000 , 
                   tickPause = 0.25 , 
                   breakOnFailure = 1 , breakOnSuccess = 1 , 
                   Nverb = 100 )

### RUN BT Unholster_Drill ###

M4 checked OUT!
Root node Unholster_Drill succeeded!

[-] Unholster_Drill [o]
    --> Move_Arm [o]
    --> Move_Arm [o]
    --> Set_Grip_Torque [o]
    --> Set_Fingers [o]
    --> Move_Arm [o]
    --> Set_Fingers [o]
    --> Move_to_Contact [o]
    --> Set_Grip_Torque [o]
    [o] Recover_or_Quit [o]
        -^- Run_to_X_Failures_DECO [o]
            [o] Attached_Recover [o]
                --> COND_Drill_Attached [o]
                [-] Recovery_Sequence [-]
                    --> Set_Fingers [-]
                    --> Set_Grip_Torque [-]
                    --> Move_Arm [-]
                    --> Set_Fingers [-]
                    --> Move_to_Contact [-]
                    --> Set_Grip_Torque [-]
                    --> COND_Drill_Attached [-]
        [-] Give_Up [-]
            --> Set_Fingers [-]
            --> Move_Arm [-]
            --> Set_Fingers [-]
    [-] Got_It [o]
        --> COND_Drill_Attached [o]
        --> Check_OUT_Drill [o]
    

## Jog, Lightly Tap with Drill, return Drill
`Jog_Safe` is a behavior that traverses the hand at a safe height `zSAFE` to a pose 

In [20]:
tapTree = Sequence( memory = 1 ) 

# 1. Traverse to a position above the probe location
tapTree.add_child(
    Jog_Safe( aboveTableDrill , # Destination
              zSAFE = _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
              hover = 1 , # Flag: remain at `zSAFE` instead of descending
              ctrl  = robot  )
)

# 2. Approach the table at a **gentle speed** and stop when a small reaction force is observed 
tapTree.add_child(
    Move_to_Contact( 
        Fmag      = 3.0 , # Reaction force that triggers stop, Forces below 1.5N aren't reliably detected
        relMove   = [0.0, 0.0, -1.0] , # Specify a move the goes *beyond* where you expect to end up
        biasWrist = 1 , # Bias wrist before beginning motion (Sorry, currently this takes 1-8 seconds)
        mode      = 'l' , 
        speed     = 0.040 , accel = 0.04 , # **gentle speed**
        pull      = 0 , # For pulling actions, reaction force tugs on hand rather than pushes
        ctrl      = robot
    ) # NOTE: This behav. may be used for absolute pose if an argument for `pose` is provided
)

# 3. Back up
tapTree.add_child(
    Move_Arm_Relative( 
        translation = [ 0.0, 0.0, 0.020 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        ctrl = robot
    )
)

# 4. set_drill_torque(200)
tapTree.add_child(  Set_Drill_Torque( drillTorque = 200 , ctrl = robot )  )

# 4.1. Activate Drill
tapTree.add_child(
    Turn_Drill_CW( ctrl = robot )
)

# 5. Maintain Pressure for Dwell Time (Assumes vertical pressure)
tapTree.add_child(
    Maintain_Z_Pressure( 
        timeOut_s = 5.0 , # Time to dwell [s]
        plungeMove = 1 , # Flag for move-to-contact
        plungeDepth=0.050 , # Distance to move down
        descendSpeed=0.030 , 
        zPress=20.0 , # [N] Pressure to maintain, ()
        maxZStep=0.010 , biasWrist=1 ,
        ctrl = robot
    )
)

# 6. Reverse drill to avoid embarrassing loss of bit
tapTree.add_child(
    Turn_Drill_CCW( ctrl = robot )
)

# 7. Back up
tapTree.add_child(
    Move_Arm_Relative( 
        translation = [ 0.0, 0.0, 0.040 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        speed     = 0.060 , accel = 0.06 , # **gentle speed**
        ctrl = robot
    )
)

# 8. Stop drill
tapTree.add_child(
    Halt_Drill( ctrl = robot )
)

# 8. Return drill
tapTree.add_child(
    Holster_Drill( 
        M4_GraspPose , 
        'M4' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
        driverLen = 0.160 ,
        ctrl = robot 
    )
)

UUID('dba1d47e-f4ba-4ee5-9a0b-c57e0294edfe')

In [21]:
run_BT_until_done( tapTree )

### RUN BT Sequence ###

def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 9.249039888381958 seconds!
Halt_Drill: Ended with drill ATTACHED!
Root node Sequence succeeded!

[-] Sequence [o]
    [-] Jog_Safe [o]
        --> Move_Arm [o]
        --> Move_Arm [o]
    --> Move_to_Contact [o]
    --> Move_Arm_Relative [o]
    --> Set_Drill_Torque [o]
    --> Turn_Drill_CW [o]
    --> Maintain_Z_Pressure [o]
    --> Turn_Drill_CCW [o]
    --> Move_Arm_Relative [o]
    --> Halt_Drill [o]
    [-] Holster_Drill [o]
        [-] COND_Holding_Drill [o]
            --> COND_Drill_Checked_Out [o]
            --> COND_Drill_Attached [o]
        --> COND_Drill_Attached [o]
        [-] Jog_Safe [o]
            --> Move_Arm [o]
            --> Move_Arm [o]
        --> Move_Arm [o]
        --> Move_Arm [o]
        --> Move_Arm [o]
        --> Move_to_Contact [o]
        --> Set_Grip_Torque [o]
        --> Set_Fingers [o]
   

## Test M6

In [22]:
testTree = Sequence( memory = 1 ) 

testTree.add_child(
    Unholster_Drill( 
        M6_GraspPose , 
        'M6' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, zClose=0.040, zRetry=0.030 , Tpliant = 0.3 , 
        wdthNarrow = 0.038 , maxIter=5, drillPress = _DRILL_CONTACT_PRESS , 
        zFree= 0.200 , 
        ctrl = robot 
    )
)

testTree.add_child(
    Holster_Drill( 
        M6_GraspPose , 
        'M6' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= 0.200 , 
        driverLen = 0.160 ,
        ctrl = robot 
    )
)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
              mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
              speed = 0.125 , 
              accel = 0.35 , 
              ctrl  = robot )
)

run_BT_until_done( testTree )

### RUN BT Sequence ###

M6 checked OUT!
Root node Sequence succeeded!

[-] Sequence [o]
    [-] Unholster_Drill [o]
        --> Move_Arm [o]
        --> Move_Arm [o]
        --> Set_Grip_Torque [o]
        --> Set_Fingers [o]
        --> Move_Arm [o]
        --> Set_Fingers [o]
        --> Move_to_Contact [o]
        --> Set_Grip_Torque [o]
        [o] Recover_or_Quit [o]
            -^- Run_to_X_Failures_DECO [o]
                [o] Attached_Recover [o]
                    --> COND_Drill_Attached [o]
                    [-] Recovery_Sequence [-]
                        --> Set_Fingers [-]
                        --> Set_Grip_Torque [-]
                        --> Move_Arm [-]
                        --> Set_Fingers [-]
                        --> Move_to_Contact [-]
                        --> Set_Grip_Torque [-]
                        --> COND_Drill_Attached [-]
            [-] Give_Up [-]
                --> Set_Fingers [-]
                --> Move_Arm [-]
                --> Set_

# Point Clouds

In [23]:
from cloud_proc.cloud_pipeline import Cloud_Pipeline
CPL = Cloud_Pipeline()

Cloud_Pipeline object created from components:  [<class 'cloud_proc.cloud_capturing.Cloud_Capturing'>, <class 'cloud_proc.cloud_poses.Cloud_Poses'>, <class 'cloud_proc.cloud_segmentation.Cloud_Segmentation'>, <class 'cloud_proc.cloud_processing.Cloud_Processing'>, <class 'cloud_proc.viewer_threejs.Viewer'>]


In [24]:
robot.arm.move_speed( SAFE_POSE , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )
robot.hand.set_finger_torque(0.3)
robot.hand.set_finger_width(1.0)

True

In [25]:
robot.camera.set_disparity_shift_dist( SAFE_POSE[2,3] - 0.030 ) # Fetch Z
# Capture pnt Cloud and transform to lab frame
cloudInLabFrame = transform_points( robot.camera.get_cloud() , robot.get_base_to_camera_pose() )

set_disparity_shift_dist: Got distance: 0.2557657378789513 , Apply Offset: -0.079


In [26]:
view = CPL.PC_Viewer()
view.add_cloud( cloudInLabFrame )
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.29025132362526374, 1.0650937462534142, 3.0…

# Utils & Setup

In [27]:
robot.hand.deactivate_motors()

1

In [28]:
robot.arm.get_tcp_pose()

array([[-9.99999059e-01,  1.37103935e-03,  3.72023956e-05,
        -9.15039339e-02],
       [ 1.37109829e-03,  9.99997726e-01,  1.63343855e-03,
        -2.49352707e-01],
       [-3.49628024e-05,  1.63348803e-03, -9.99998665e-01,
         2.85757773e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [5]:
robot.hand.set_finger_torque(0.3)

True

In [6]:
robot.hand.set_finger_width(0.0)

True

In [2]:
robot.hand.deactivate_motors()

1

In [18]:
# M4_GraspPose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
# %store M4_GraspPose

In [19]:
robot.arm.move_speed( midDrillPose , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

True

In [20]:
# midDrillPose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
# %store midDrillPose

In [21]:
# aboveTableDrill = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
# %store aboveTableDrill

In [22]:
robot.arm.move_speed( aboveTableDrill , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

True

# Run this after teaching any poses

In [23]:
import pickle , traceback

save_names_to_PKL( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'startPose' ,
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' , 
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)

{'nameStringList': ['startPose', 'M8_GraspPose', 'M6_GraspPose', 'M4_GraspPose', 'GearShaft1_Setdown', 'GearShaft2_Setdown', 'BNC_Ma_Setdown', 'KeySm_Setdown', 'KeyLg_Setdown', 'ShaftSm_Setdown', 'ShaftLg_Setdown', 'KeyLg_Grasp', 'ShaftLg_Grasp', 'BNC_Fe_Grasp', 'ShaftSm_Grasp', 'GearLg_Grasp', 'GearLg_Grasp', 'Ether_setdown', 'M8_1_mag', 'M8_2_mag', 'M8_3_mag', 'M8_4_mag', 'M8_5_mag', 'M8_6_mag', 'M6_1_mag', 'M6_2_mag', 'M6_3_mag', 'M6_4_mag', 'M6_5_mag', 'M6_6_mag', 'M4_1_mag', 'M4_2_mag', 'M4_3_mag', 'M4_4_mag', 'M4_5_mag', 'M4_6_mag', 'releasePose', 'SAFE_POSE', 'midDrillPose', '_LRG_GEAR_DEX', '_SML_GEAR_DEX', 'aboveTableDrill'], 'pklStruct': {}, 'pklPath': 'taughtPoses.pkl'}
Pickling taughtPoses.pkl... Success!
### Completed Writing taughtPoses.pkl ###
